<a href="https://colab.research.google.com/github/TXH2020/MainRepo/blob/main/Academics/Image%20Processing/AttProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade -q gspread
!pip install face_recognition
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 23 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=ad586c7602cfc5a95c4e29f5c3d4938c8edfa871ef7aca3ad242c0465516d627
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.3-py3-none-any.whl size=14840 sha256=efdd89f54cc644f6c20ab9051d83451d332d4cc3cae534e4e5c44905e0233bc3
  Stored in directory: /tmp/pip-ephem-wheel-cache-_uow673d/wheels/94/8d/0b/bdcd83555c3555f91a33f6c2384428d9f163c7d75ab0d272b4
Su

In [2]:
import numpy as np
import gspread
import pandas as pd
from google.colab import auth
from google.colab import drive
from google.auth import default
import pickle
import os
import gdown

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
worksheet_name='Database Of Face Photos (Responses)'
worksheet = gc.open(worksheet_name).sheet1
rows = worksheet.get_all_values()
df=pd.DataFrame.from_records(rows,columns=rows[0],nrows=1)
df1=df.drop(0)
df1.reset_index
df1.set_index(np.arange(0,len(rows)-1),inplace=True)
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import face_recognition
from PIL import Image
# Load the jpg files into numpy arrays
def encoding(df,face_encoding):
  known_image=[]
  for i in range(df.shape[0]):
    known_image.append(face_recognition.load_image_file(df['NAME'][i]+".jpg"))

# Get the face encodings for each face in each image file
# Since there could be more than one face in each image, it returns a list of encodings.
# But since I know each image only has one face, I only care about the first encoding in each image, so I grab index 0.
  for i in range(df.shape[0]):
    try:
      print(i)
      face_encoding.append(face_recognition.face_encodings(known_image[i],model='cnn')[0])
  #unknown_face_encoding = face_recognition.face_encodings(unknown_image,model='cnn')
    except IndexError:
      im1=Image.open(df['NAME'][i]+".jpg")
      im2=im1.copy().rotate(270)
      im2.save('check.jpg')
      error_image=face_recognition.load_image_file("check.jpg")
      try:
        face_encoding.append(face_recognition.face_encodings(error_image,model='cnn')[0])
      except:
        im1=Image.open('check.jpg')
        im2=im1.copy().rotate(180)
        im2.save('check1.jpg')
        error_image1=face_recognition.load_image_file("check1.jpg")
        face_encoding.append(face_recognition.face_encodings(error_image1,model='cnn')[0])
  store_data(face_encoding)

  #l=[0 for i in range(len(face_encoding))]
"""for i in range(len(unknown_face_encoding)):
  result=face_recognition.compare_faces(face_encoding, unknown_face_encoding[i],tolerance=0.54)
  if(True in result):
    l[result.index(True)]=1
for i in range(len(l)):
  if(l[i]==1):
    print(known_names[i], " is Present")
  else:
    print(known_names[i], " is Absent")"""

'for i in range(len(unknown_face_encoding)):\n  result=face_recognition.compare_faces(face_encoding, unknown_face_encoding[i],tolerance=0.54)\n  if(True in result):\n    l[result.index(True)]=1\nfor i in range(len(l)):\n  if(l[i]==1):\n    print(known_names[i], " is Present")\n  else:\n    print(known_names[i], " is Absent")'

In [4]:
def download_photos(df,l1):
 for i in range(df.shape[0]):
  print(i)
  a=df['Your Photo'][i]
  file_id=a[a.find('=')+1:]
  url="https://drive.google.com/uc?id={}".format(file_id)
  gdown.download(url, df['NAME'][i]+".jpg", quiet=True)
 encoding(df,l1)

In [5]:
def store_data(face_encoding):
  %cd drive/MyDrive
  with open(worksheet_name+'.pkl','wb') as p:
    pickle.dump(face_encoding,p)
  %cd /content

In [6]:
if(os.path.isfile('drive/MyDrive/'+worksheet_name+'.pkl')==False):
  download_photos(df1,[])

%cd drive/MyDrive
with open(worksheet_name+'.pkl','rb') as p:
    face_encoding=pickle.load(p)
%cd /content

if(len(face_encoding)<df1.shape[0]):
  download_photos(df1.iloc[len(face_encoding):],face_encoding)

/content/drive/MyDrive
/content
